<a href="https://colab.research.google.com/github/gsvenky89/Assignemnt_2_ML/blob/master/CNN_for_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install keras-tqdm

In [0]:
def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s

In [0]:
import sys
sys.path.append("..")
sys.setrecursionlimit(3000)

In [59]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print("TensoreFlow Ver:",tf.__version__)
print("Keras Ver:",keras.__version__)
import keras.utils
from keras_tqdm import TQDMCallback
import cv2
from sklearn.metrics import accuracy_score

TensoreFlow Ver: 1.12.0
Keras Ver: 2.2.4


In [60]:
reset_tf_session() #clearing up any default graph

In [0]:
from keras.datasets import cifar10
(x_train_i, y_train_i), (x_test_i, y_test_i) = cifar10.load_data()
x_train = x_train_i[range(5000),:]
y_train = y_train_i[range(5000),:]
x_test = x_test_i[range(2000),:]
y_test = y_test_i[range(2000),:]
del x_train_i,y_train_i,x_test_i,y_test_i

In [62]:
print("Train samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

Train samples: (5000, 32, 32, 3) (5000, 1)
Test samples: (2000, 32, 32, 3) (2000, 1)


In [0]:
NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

In [64]:
# normalize inputs
x_train2 = x_train/255. - 0.5
x_test2 = x_test/255. - 0.5
del x_train
del x_test
# convert class labels to one-hot encoded, should have shape (?, NUM_CLASSES)
y_train2 = keras.utils.to_categorical(y_train[range(5000),:], num_classes=NUM_CLASSES)
y_test2 = keras.utils.to_categorical(y_test[range(2000),:], num_classes=NUM_CLASSES)
print(x_test2[600,:,:,:])
print(y_train2.shape,y_test2.shape)

[[[-0.27647059 -0.09215686  0.09215686]
  [-0.27647059 -0.09215686  0.08823529]
  [-0.26862745 -0.08431373  0.09607843]
  ...
  [-0.28039216 -0.10784314  0.06862745]
  [-0.28431373 -0.11568627  0.06470588]
  [-0.28431373 -0.11568627  0.06078431]]

 [[-0.26078431 -0.07647059  0.10784314]
  [-0.26078431 -0.07647059  0.1       ]
  [-0.25686275 -0.07254902  0.10784314]
  ...
  [-0.26862745 -0.09607843  0.08039216]
  [-0.27254902 -0.1         0.07254902]
  [-0.27254902 -0.1         0.07254902]]

 [[-0.25294118 -0.05294118  0.12745098]
  [-0.25294118 -0.05686275  0.11960784]
  [-0.25294118 -0.05294118  0.12352941]
  ...
  [-0.26078431 -0.07647059  0.08823529]
  [-0.26078431 -0.08039216  0.08823529]
  [-0.26078431 -0.08823529  0.08039216]]

 ...

 [[ 0.0372549   0.06862745  0.04901961]
  [-0.06078431 -0.02941176 -0.04901961]
  [-0.11568627 -0.08431373 -0.10392157]
  ...
  [-0.36666667 -0.34705882 -0.37058824]
  [-0.41372549 -0.39411765 -0.41764706]
  [-0.43333333 -0.40980392 -0.4372549 ]]

 [

In [0]:
# import necessary building blocks
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add, Dropout
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras import initializers
from keras.engine import Layer, InputSpec
from keras.layers.advanced_activations import LeakyReLU

In [0]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()
    
    # CONV 1
    # first layer needs to define "input_shape"
    model.add(Conv2D(16, (3, 3), strides = (1, 1), padding="same", name = 'conv_layer_1', input_shape=(32, 32, 3)))   
    model.add(LeakyReLU(0.1))
    
    # CONV 2
    model.add(Conv2D(32, (3, 3), strides = (1, 1), padding="same", name = 'conv_layer_2'))  
    model.add(LeakyReLU(0.1))
    
    # MaxPooling2D 1
    model.add(MaxPooling2D((2, 2), name='max_pool_1'))
    
    # Dropout
    model.add(Dropout(0.25, noise_shape=None, seed=0))
    
    # CONV 3
    model.add(Conv2D(32, (3, 3), strides = (1, 1), padding="same", name = 'conv_layer_3')) 
    model.add(LeakyReLU(0.1))
    
    # CONV 4
    model.add(Conv2D(64, (3, 3), strides = (1, 1), padding="same", name = 'conv_layer_4'))  
    model.add(LeakyReLU(0.1))
    
    # MaxPooling2D 1
    model.add(MaxPooling2D((2, 2), name='max_pool_2'))
    
    # Dropout
    model.add(Dropout(0.25, noise_shape=None, seed=0))
    
    # Flatten
    model.add(Flatten())    
    
    # FC
    model.add(Dense(256, name='fc1'))
    model.add(Dropout(0.5, noise_shape=None, seed=0))
    
    # FC
    model.add(Dense(NUM_CLASSES))  # the last layer with neuron for each class    
    model.add(Activation("softmax"))  # output probabilities

    return model


In [67]:
# describe model
reset_tf_session() # clear default graph
model = make_model()
model.summary()
model.compile(
    loss='categorical_crossentropy', # this is our cross-entropy
    optimizer='adam',
    metrics=['accuracy']  # report accuracy during training
)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_layer_1 (Conv2D)        (None, 32, 32, 16)        448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 16)        0         
_________________________________________________________________
conv_layer_2 (Conv2D)        (None, 32, 32, 32)        4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pool_1 (MaxPooling2D)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv_layer_3 (Conv2D)        (None, 16, 16, 32)        9248      
__________

In [68]:
model.fit(
    x_train2, 
    y_train2,
    batch_size=8, 
    epochs=10,
    validation_split= 0.3,
    #validation_data=(x_test2, y_test2),
    verbose = 1,shuffle=True
)

Train on 3500 samples, validate on 1500 samples
Epoch 1/10
3500/3500 [==============================] - 6s 2ms/step - loss: 1.9359 - acc: 0.3043 - val_loss: 1.6199 - val_acc: 0.4173
Epoch 2/10
3500/3500 [==============================] - 6s 2ms/step - loss: 1.5630 - acc: 0.4346 - val_loss: 1.4398 - val_acc: 0.4813
Epoch 3/10
3500/3500 [==============================] - 6s 2ms/step - loss: 1.3882 - acc: 0.5063 - val_loss: 1.4328 - val_acc: 0.4827
Epoch 4/10
3500/3500 [==============================] - 6s 2ms/step - loss: 1.2100 - acc: 0.5737 - val_loss: 1.3867 - val_acc: 0.5127
Epoch 5/10
3500/3500 [==============================] - 6s 2ms/step - loss: 1.0908 - acc: 0.6097 - val_loss: 1.3098 - val_acc: 0.5307
Epoch 6/10
3500/3500 [==============================] - 6s 2ms/step - loss: 0.9407 - acc: 0.6660 - val_loss: 1.3079 - val_acc: 0.5547
Epoch 7/10
3500/3500 [==============================] - 6s 2ms/step - loss: 0.8279 - acc: 0.7046 - val_loss: 1.3492 - val_acc: 0.5607
Epoch 8/10
350

In [69]:
y_pred_test = model.predict(x_test2)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
y_pred_test_max_probas = np.max(y_pred_test, axis=1)
print("Test accuracy:", accuracy_score(y_test, y_pred_test_classes))

Test accuracy: 0.5595


In [0]:
model.save_weights("weights.h5")